In [94]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
#NASA Mars News
url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
browser.visit(url)

In [27]:
newshtml = browser.html
soup = bs(newshtml, 'html.parser')
page = soup.find_all("div", {"id": "page"})
page

[<div id="page">
 <!-- title to go in the page_header -->
 <div class="header_mask">
 <section class="content_page module"></section>
 </div>
 <div class="grid_list_page module content_page">
 <div class="grid_layout">
 <article>
 <header id="page_header">
 </header>
 <div class="react_grid_list" data-react-class="GridListPage" data-react-props='{"left_column":false,"class_name":"","default_view":"list_view","model":"news_items","view_toggle":false,"search":"true","list_item":"News","title":"News","categories":["19,165,184,204"],"order":"publish_date desc,created_at desc","no_items_text":"There are no items matching these criteria.","site_title":"NASA’s Mars Exploration Program ","short_title":"Mars","site_share_image":"/system/site_config_values/meta_share_images/1_142497main_PIA03154-200.jpg","per_page":null,"filters":"[ [ \"date\", [ [ \"2019\", \"2019\" ], [ \"2018\", \"2018\" ], [ \"2017\", \"2017\" ], [ \"2016\", \"2016\" ], [ \"2015\", \"2015\" ], [ \"2014\", \"2014\" ], [ \"201

In [29]:
#test = soup.find_all("h3")
#for each in test:
#    print(each)

#above works because find will only find the first one, which is the most recent -- same for the body, below
news_title = soup.find("h3").text
news_title

'InSight Is the Newest Mars Weather Service'

In [33]:
news_p = soup.find("div",class_="article_teaser_body").text
news_p

"By collecting data around the clock, NASA's lander will provide unique science about the Martian surface."

In [48]:
#JPL Mars Space Images - Featured Image

url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)

In [49]:
jplhtml = browser.html
soup = bs(jplhtml, 'html.parser')

In [52]:
#carouselPic = soup.find('article',class_='carousel_item')
#image url is inside an article tag in the 'style' parameter, which is why
#we're accessing it below like so
imageExt = soup.article['style'][23:-3]
featured_image_url = 'https://www.jpl.nasa.gov' + imageExt
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA19101-1920x1200.jpg'

In [58]:
#Mars Weather
url = "https://twitter.com/marswxreport?lang=en"
browser.visit(url)

In [60]:
twitterhtml = browser.html
soup = bs(twitterhtml, 'html.parser')

In [62]:
page = soup.find()
page

<html data-scribe-reduced-action-queue="true" lang="en" xmlns="http://www.w3.org/1999/xhtml"><head>
<meta charset="utf-8"/>
<script async="" src="//www.google-analytics.com/analytics.js"></script><script nonce="">
        !function(){window.initErrorstack||(window.initErrorstack=[]),window.onerror=function(r,i,n,o,t){r.indexOf("Script error.")&gt;-1||window.initErrorstack.push({errorMsg:r,url:i,lineNumber:n,column:o,errorObj:t})}}();
      </script>
<script id="bouncer_terminate_iframe" nonce="">
    if (window.top != window) {
  window.top.postMessage({'bouncer': true, 'event': 'complete'}, '*');
}
  </script>
<script id="ttft_boot_data" nonce="">
    window.ttftData={"transaction_id":"008f93360035f986.a4d1d0353a1dc39f\u003c:00e8a8d8009b4995","server_request_start_time":1550722474441,"user_id":null,"is_ssl":true,"rendered_on_server":true,"is_tfe":true,"client":"macaw-swift","tfe_version":"tsa_b\/1.0.1\/20190206.2140.12b8a68","ttft_browser":"chrome"};!function(){function t(t,n){window.

In [63]:
tweets = soup.find_all("p", class_ = "TweetTextSize TweetTextSize--normal js-tweet-text tweet-text")

In [87]:
#Get the first tweet
count = 1
for tweet in tweets:
    #make sure it's a weather tweet
    if "pressure at" in tweet.text or "winds from the" in tweet.text:
        startIdx = tweet.text.lower().index("sol")
        if "\n" in tweet.text:
            endIdx = tweet.text.index("\n")
            print(tweet.text[startIdx:endIdx])
            if count == 1:
                mars_weather = tweet.text[startIdx:endIdx]
                count +=1
        else:
            print(tweet.text[startIdx:])
            if count == 1:
                mars_weather = tweet.text[startIdx:]
                count +=1
mars_weather

sol 81 (2019-02-17), high -17/2F, low -95/-138F, pressure at 7.23hPa, winds from the WNW at 12 mph gusting to 37.8 mph
sol 80 (2019-02-16), high -16/3F, low -95/-139F, pressure at 7.23hPa, winds from the WNW at 10.7 mph gusting to 32.3 mph
sol 79 (2019-02-15), high -13/8F, low -95/-138F, pressure at 7.21hPa, winds from the WNW at 9.9 mph gusting to 26.2 mph
sol 78 (2019-02-14), high -15/4F, low -95/-138F, pressure at 7.22hPa, winds from the WNW at 10.5 mph gusting to 36.4 mph
sol 77 (2019-02-13), high -13C/10F, low -94C/-136F, pressure at 7.21 hPa, winds from the WNW at 9 mph gusting to 26.6 mph
Sol 2319 (2019-02-13), high -17C/1F, low -72C/-97F, pressure at 8.12 hPa, daylight 06:46-18:52pic.twitter.com/anlHR95BMs
Sol 2318 (2019-02-12), high -17C/1F, low -73C/-99F, pressure at 8.11 hPa, daylight 06:46-18:52
Sol 2317 (2019-02-11), high -16C/3F, low -73C/-99F, pressure at 8.12 hPa, daylight 06:46-18:52pic.twitter.com/D15lfEs6RN
Sol 2316 (2019-02-10), high -14C/6F, low -72C/-97F, pressure

'sol 81 (2019-02-17), high -17/2F, low -95/-138F, pressure at 7.23hPa, winds from the WNW at 12 mph gusting to 37.8 mph'

In [88]:
#Mars Facts
url = "https://space-facts.com/mars/"
browser.visit(url)

In [90]:
factshtml = browser.html
soup = bs(factshtml, 'html.parser')

In [117]:
table = soup.find_all("table", {'id':'tablepress-mars'})

io = ""
for item in table:
    io = io + str(item)
    
marslist = pd.read_html(io)
#choose the dataframe from list of dataframes
df = marslist[0]
#rename the '0' and '1' columns with meaningful titles
df = df.rename(columns = {0:'Description', 1:'Value'}).set_index('Description')

#make an html string to represent the dataframe
dfhtml = df.to_html()

df

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [118]:
#Mars Hemispheres
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

In [119]:
hemhtml = browser.html
soup = bs(hemhtml, 'html.parser')

In [124]:
imgUrls = []
links = soup.find_all("div", class_="item")
for link in links:
    imglink = "https://astrogeology.usgs.gov" + link.a['href']
    imgUrls.append(imglink)
imgUrls

['https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced']

In [145]:
#visit each of the image links to get the image url and hemisphere name
hemisphere_image_urls = []
for idx, val in enumerate(imgUrls):
    url = val
    browser.visit(url)
    
    imghtml = browser.html
    soup = bs(imghtml, 'html.parser')
    
    imgs = soup.find_all("img", class_ = "wide-image")
    name = soup.find("h2",class_="title")
    
    for content in imgs:
        img_url = 'https://astrogeology.usgs.gov' + content["src"]
    
    #append a dictionary to list of dicts for each image
    hemisphere_image_urls.append({'title': name.text[:-9], 'img_url' : img_url})
    
    #print what the contents of the dictionary will look like
    print(name.text[:-9])
    print(img_url)
    print("--------------------------------------------------------")

Cerberus Hemisphere
https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg
--------------------------------------------------------
Schiaparelli Hemisphere
https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg
--------------------------------------------------------
Syrtis Major Hemisphere
https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg
--------------------------------------------------------
Valles Marineris Hemisphere
https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg
--------------------------------------------------------


In [147]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]